<a href="https://colab.research.google.com/github/GAYARRI/text-classification/blob/master/GPT4_ampliado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai
!pip install python-dotenv
!pip install google-cloud-bigquery
from google.cloud import bigquery
from google.oauth2 import service_account
credentials = service_account.Credentials.from_service_account_file('google_bigquery_credential.json')
client = bigquery.Client(credentials=credentials)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


FileNotFoundError: ignored

In [ ]:
import openai
import os
import pandas as pd
import json

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [ ]:
def get_completion(prompt, model="gpt-4-0314"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

In [ ]:
def setprompt(TIPO_TRANSPARENCIA,titulo,texto,preguntas,textoRespuestaslibres,textofillblanks,origenes,destinos,conceptos,NAME_TIPO_TRANSPARENCIA,subject="History"):

    prompt = f"""We need to define exactly fiveteen most relevant one-word translated to english topics space separated, \
    in lower case, deleting accent marks and free of special caracters, for exercises carried out by \
    with the intention of creating groupings that allow us to relate the exercises to each other. \

    The subject of the exercice is '''{subject}''' \

    In this exercise the student has to '''{NAME_TIPO_TRANSPARENCIA}''' \
    """

    if title!="":
      prompt+=f"""
        The title of the exercice is '''{titulo}''' \
      """
    prompt+=f"""
      The text of the exercice is '''{texto}''' \
    """

    if tiposlide==10:
      prompt+=f"""
        The items in column A are '''{origenes}''' \
        The items in column B are '''{destinos}''' \
     """
     if tiposlide==[6,18]:
      prompt+=f"""
        The fillFillblanks response are '''{textofillblanks}''' \
     """
     if tiposlide==[2,4]:
      prompt+=f"""
        The questions of the exercise are '''{preguntas}''' \
     """
     if tiposlide==12:
      prompt+=f"""
        The concepts of the exercise are '''{conceptos}''' \
     """
     if tiposlide==11:
      prompt+=f"""
        The free answer is '''{textoRespuestaslibres}''' \
     """
     if tiposlide==13:
      prompt+=f"""
        The free answer is '''{conceptos}+{grupos}''' \
     """
     if tiposlide==19:
      prompt+=f"""
        The test questions are '''{preguntas}''' \
     """


    prompt+="""
    Make each topic exactly one words long and format your response in JSON format.\
    This is a list of stopwords you must strictly avoid in the output ot this prompt:\

    ["History","history","assesment","assessment","autoevaluation","evaluation","evaluacion","evaluación","selfassesment","self-assesment","selfevaluation","self-evaluation","test","answers","answer",\
    "classify","concepto","concept","actividad","activitat","response","responses","respuesta","respuestas","relacionar","elegir","marcar","identificar","identify"]

    If one of this , appear in the final output, please change it for another one not in the list .
    For the final output we choose only ten elegible topics
    """

    print(prompt)
    response = get_completion(prompt)
    #print (response)
    return response


In [ ]:
query = """
    SELECT * FROM projectml-345114.recommender_data.EtiquetadoEditex where idtransparencia in () order by idtransparencia
"""

query_job = client.query(query)
results = query_job.result().to_dataframe()
dftopics = pd.DataFrame()

for index, row in results.iterrows():
    tiposlide=row['TIPO_TRANSPARENCIA']
    title = row['titulo']
    question = row['texto']
    answers = row['preguntas']
    origenes = row['origenes']
    textofillblanks = row['preguntas']
    task=""
    destinos = row['destinos']

    if tiposlide==10:
      task="relate concepts"

    data=json.loads(setprompt(tiposlide,title,question,answers,textofillblanks,origenes,destinos,task))

    topics = data['topics']
    print(topics)


NameError: ignored